In [1]:
import pypsa
import pandas as pd
import yaml
import os


In [2]:
def create_baseline_network(config_file, params_file, data_path):
    """
    Builds the baseline network for the Social Building scenario in the Lyngby PED project.
    """

    # Load Config and Parameters 
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    with open(params_file, 'r') as f:
        params = yaml.safe_load(f)

    # Create an empty PyPSA Network
    network = pypsa.Network()

    # Set Snapshots (Hourly for 1 year)
    snapshots = pd.date_range(start="2025-01-01", periods=8760, freq="H")
    network.set_snapshots(snapshots)

    # Add Buses 
    network.add("Bus", "Electricity Bus", carrier="electricity")
    network.add("Bus", "Heat Bus", carrier="heat")
    # Definizione carrier con emissioni di CO2 associate al calore
    network.add("Carrier", "heat", co2_emissions=0.2)  # 0.2 tCO₂/MWh_th per caldaia a gas

    network.add("Generator",
            name="Heat Source",
            bus="Heat Bus",
            carrier="heat",  # deve essere "heat" per legarsi all’emissione
            p_nom_extendable=True,
            marginal_cost=50)

    # Load Input Data
    loads = pd.read_csv(os.path.join(data_path, "Social_Building_Loads.csv"), index_col=0, parse_dates=True)
    pv_generation = pd.read_csv(os.path.join(data_path, "Solar_PV_Generation.csv"), index_col=0, parse_dates=True)
    grid_import_cost = params["grid"]["import_cost_eur_per_mwh"]


    # Check colonne disponibili
    expected_columns = ["electricity_demand", "heat_demand"]
    available_columns = loads.columns.str.strip().str.lower()

    missing_columns = [col for col in expected_columns if col not in available_columns]
    if missing_columns:
        print(f"Errore: mancano le colonne seguenti nel file Social_Building_Loads.csv: {missing_columns}")
        print(f"Colonne trovate: {list(loads.columns)}")
        raise KeyError(f"Colonne mancanti nel file CSV: {missing_columns}")
    else:
        print("Tutte le colonne richieste sono presenti.")

    # Normalizza i nomi colonna
    loads.columns = available_columns  # .str.strip().str.lower() già fatto sopra

    # Add Electrical Load
    network.add("Load",
            name="Social Building Electricity Load",
            bus="Electricity Bus",
            carrier="electricity",
            p_set=loads["electricity_demand"])

    # Add Heat Load
    network.add("Load",
            name="Social Building Heat Load",
            bus="Heat Bus",
            carrier="heat",
            p_set=loads["heat_demand"])


    # Add Existing PV Generator
    network.add("Generator",
                name="Existing PV",
                bus="Electricity Bus",
                carrier="solar",
                p_nom_extendable=False,
                p_nom=pv_generation["solar_pv_generation"].max(),  # Peak PV capacity
                p_max_pu=pv_generation["solar_pv_generation"] / pv_generation["solar_pv_generation"].max(),
                marginal_cost=0)


    network.add("Generator",
                name="Grid",
                bus="Electricity Bus",
                carrier="electricity",
                p_nom_extendable=True,
                marginal_cost=grid_import_cost)

    # Add Heat Source Generator:sto creando un generatore interno chiamato "Heat Source". Questo rappresenta una fonte locale di produzione di calore
    heat_source_type = params["social_building"].get("heat_source", "boiler")

    if heat_source_type == "boiler":
        marginal_cost = params["social_building"].get("boiler_marginal_cost", 50)  # €/MWh
    elif heat_source_type == "district_heating":
        marginal_cost = params["social_building"].get("district_heating_marginal_cost", 30)  # €/MWh
    else:
        raise ValueError(f"Unknown heat source type: {heat_source_type}")


    return network


In [3]:
# Step 1 - Percorsi corretti
config_file = "config.yml"
params_file = "component_params.yml"
data_path = "../data/input/timeseries"
import pandas as pd

# Percorsi dei file
elec_file = "../data/input/timeseries/Electricity_Demand_General.csv"
heat_file = "../data/input/timeseries/Social_Building_Loads.csv"
output_file = "../data/input/timeseries/Social_Building_Loads.csv"  # sovrascrive

# Carica i due file
electricity = pd.read_csv(elec_file, index_col=0, parse_dates=True)
heat = pd.read_csv(heat_file, index_col=0, parse_dates=True)

# Normalizza nomi colonne
electricity.columns = ["electricity_demand"]
# Assumiamo che il file sia stato caricato come DataFrame con una sola colonna
heat = pd.read_csv("../data/input/timeseries/Heat_Demand_General.csv", index_col=0, parse_dates=True)

# Rinomina correttamente se ha solo 1 colonna
if heat.shape[1] == 1:
    heat.columns = ["heat_demand"]
else:
    print("Il file contiene più colonne, specificare quale usare.")

# Unione per indice temporale
merged = pd.concat([electricity, heat], axis=1)

# Salva il nuovo file
merged.to_csv(output_file)

print("File unito salvato con successo.")
print(merged.head())



# Step 2 - Creazione rete
network = create_baseline_network(config_file, params_file, data_path)

# Step 3 - Stampa rete
print(network)


File unito salvato con successo.
                     electricity_demand  heat_demand
2025-01-01 00:00:00          140.914237   195.601750
2025-01-01 01:00:00          134.198788   198.686849
2025-01-01 02:00:00          134.606463   190.876191
2025-01-01 03:00:00          139.990397   203.731247
2025-01-01 04:00:00          138.145183   185.102361
Tutte le colonne richieste sono presenti.
PyPSA Network
Components:
 - Bus: 2
 - Carrier: 1
 - Generator: 3
 - Load: 2
Snapshots: 8760


C:\Users\aless\anaconda3\envs\PED_Lyngby_Model\Lib\site-packages\pypsa\components.py:323: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
C:\Users\aless\anaconda3\envs\PED_Lyngby_Model\Lib\site-packages\pypsa\components.py:323: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
C:\Users\aless\anaconda3\envs\PED_Lyngby_Model\Lib\site-packages\pypsa\components.py:323: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype

In [4]:
network.optimize(solver_name="highs")


C:\Users\aless\anaconda3\envs\PED_Lyngby_Model\Lib\site-packages\pypsa\optimization\constraints.py:531: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(n.loads.bus, axis=1)
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|█████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.61it/s]
INFO:linopy.io: Writing time: 0.65s
INFO:linopy.solvers:Log file at C:\Users\aless\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 26282 primals, 70082 duals
Objective: 9.89e+07
Solver model: available
Solver message: optimal

C:\Users\aless\anaconda3\envs\PED_Lyngby_Model\Lib\site-packages\pypsa\optimization\optimize.py:355: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavi

('ok', 'optimal')

In [5]:
import pandas as pd

# Grid import
grid_import = network.generators_t.p["Grid"].clip(lower=0).sum()

# PV production
pv_production = network.generators_t.p["Existing PV"].sum()

# Heat production
heat_production = network.generators_t.p["Heat Source"].sum()

# Total electricity demand
electricity_demand = network.loads_t.p_set.loc[:, network.loads.carrier == "electricity"].sum().sum()

# Total heat demand
heat_demand = network.loads_t.p_set.loc[:, network.loads.carrier == "heat"].sum().sum()

# Operational cost (already known from objective)
operational_cost = network.objective

# Heat emissions (check that the carrier has an emission factor)
if "heat" in network.carriers.index and "co2_emissions" in network.carriers.columns:
    emission_factor = network.carriers.at["heat", "co2_emissions"]  # tCO2 per MWh
else:
    emission_factor = 0.0  # default if not set

heat_emissions = heat_production * emission_factor

# Summary
summary = pd.DataFrame({
    "Grid Import [MWh]": [grid_import],
    "PV Production [MWh]": [pv_production],
    "Heat Production [MWh_th]": [heat_production],
    "Electricity Demand [MWh]": [electricity_demand],
    "Heat Demand [MWh_th]": [heat_demand],
    "Operational Cost [€]": [operational_cost],
    "CO₂ Emissions Heat [tCO₂]": [heat_emissions]
})

print(summary)


   Grid Import [MWh]  PV Production [MWh]  Heat Production [MWh_th]  \
0       1.006751e+06         39791.521364             971176.105872   

   Electricity Demand [MWh]  Heat Demand [MWh_th]  Operational Cost [€]  \
0              1.046542e+06         971176.105872          9.889635e+07   

   CO₂ Emissions Heat [tCO₂]  
0              194235.221174  


In [6]:
network.export_to_netcdf("../data/output/scenario_baseline/network_results.nc")


INFO:pypsa.io:Exported network network_results.nc has loads, carriers, buses, generators


<xarray.Dataset> Size: 1MB
Dimensions:                       (snapshots: 8760, investment_periods: 0,
                                   loads_i: 2, loads_t_p_set_i: 2,
                                   loads_t_p_i: 2, carriers_i: 1, buses_i: 2,
                                   buses_t_p_i: 2, buses_t_marginal_price_i: 2,
                                   generators_i: 3, generators_t_p_max_pu_i: 1,
                                   generators_t_p_i: 3)
Coordinates:
  * snapshots                     (snapshots) int64 70kB 0 1 2 ... 8758 8759
  * investment_periods            (investment_periods) int64 0B 
  * loads_i                       (loads_i) object 16B 'Social Building Elect...
  * loads_t_p_set_i               (loads_t_p_set_i) object 16B 'Social Buildi...
  * loads_t_p_i                   (loads_t_p_i) object 16B 'Social Building E...
  * carriers_i                    (carriers_i) object 8B 'heat'
  * buses_i                       (buses_i) object 16B 'Electricity Bus' 'Hea...
  * buses_t_p_i                   (buses_t_p_i) object 16B 'Electricity Bus' ...
  * buses_t_marginal_price_i      (buses_t_marginal_price_i) object 16B 'Elec...
  * generators_i                  (generators_i) object 24B 'Heat Source' ......
  * generators_t_p_max_pu_i       (generators_t_p_max_pu_i) object 8B 'Existi...
  * generators_t_p_i              (generators_t_p_i) object 24B 'Heat Source'...
Data variables: (12/22)
    snapshots_snapshot            (snapshots) datetime64[ns] 70kB 2025-01-01 ...
    snapshots_objective           (snapshots) float64 70kB 1.0 1.0 ... 1.0 1.0
    snapshots_stores              (snapshots) float64 70kB 1.0 1.0 ... 1.0 1.0
    snapshots_generators          (snapshots) float64 70kB 1.0 1.0 ... 1.0 1.0
    investment_periods_objective  (investment_periods) object 0B 
    investment_periods_years      (investment_periods) object 0B 
    ...                            ...
    generators_p_nom_extendable   (generators_i) bool 3B True False True
    generators_carrier            (generators_i) object 24B 'heat' ... 'elect...
    generators_marginal_cost      (generators_i) float64 24B 50.0 0.0 50.0
    generators_p_nom_opt          (generators_i) float64 24B 217.3 11.77 157.8
    generators_t_p_max_pu         (snapshots, generators_t_p_max_pu_i) float64 70kB ...
    generators_t_p                (snapshots, generators_t_p_i) float64 210kB ...
Attributes:
    network__linearized_uc:  0
    network__multi_invest:   0
    network_name:            
    network_objective:       98896350.91484904
    network_pypsa_version:   0.25.1
    network_srid:            4326
    meta:                    {}